In [72]:
import sys

sys.path.append('..')
from Poison.greedy import run_greedy

run_greedy()

tensor([False, False, False,  ..., False, False, False])
node_attrs
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
adj
SparseTensor(row=tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
             col=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
             val=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
             size=(2708, 2708), nnz=11054, density=0.15%)
labels
tensor([0, 0, 0,  ..., 0, 0, 0])
Create triples...
Done creating triples...
Making model...
Starting attack...
initial active subnetworkscore: 1.8646916462163183
--- In attack stage ---
-2 [   2.         1986.            3.76079248]
iter 1 active subnetworkscore: 1.4100335427135857
-2 [   2.         1666.            2.22937198]
iter 2 active subnetworkscore: 0.9340035729301503
Running model on poisoned data...
torch.Size([2, 0])
torch.Size([3665278, 3])
auc before poison:
0.7675661196639035
auc after poison

(<Utils.experiment_results.Experiment at 0x240edd88ee0>,
 <Utils.experiment_results.Experiment at 0x240a673f070>)

#testing with cat





In [63]:
import torch

x = torch.tensor([[],[]])

print(x)

y = torch.rand(10,3)

print(y)
print(y.shape)
y = torch.transpose(y, 0, 1)
print(y.shape)
# y_split = y[:2]
# print(y_split)

for i in range(10):
    x = torch.cat((x, y[:2, i:i+1]), dim=-1)

print(x)
print(torch.flip(x, dims=[0]))

tensor([], size=(2, 0))
tensor([[0.7104, 0.5231, 0.6517],
        [0.7096, 0.3245, 0.9439],
        [0.5093, 0.8295, 0.1232],
        [0.3547, 0.3124, 0.0160],
        [0.7869, 0.4176, 0.4849],
        [0.4530, 0.8849, 0.4267],
        [0.8729, 0.3182, 0.3065],
        [0.6186, 0.6048, 0.8804],
        [0.9447, 0.1017, 0.5970],
        [0.7629, 0.1596, 0.3819]])
torch.Size([10, 3])
torch.Size([3, 10])
tensor([[0.7104, 0.7096, 0.5093, 0.3547, 0.7869, 0.4530, 0.8729, 0.6186, 0.9447,
         0.7629],
        [0.5231, 0.3245, 0.8295, 0.3124, 0.4176, 0.8849, 0.3182, 0.6048, 0.1017,
         0.1596]])
tensor([[0.5231, 0.3245, 0.8295, 0.3124, 0.4176, 0.8849, 0.3182, 0.6048, 0.1017,
         0.1596],
        [0.7104, 0.7096, 0.5093, 0.3547, 0.7869, 0.4530, 0.8729, 0.6186, 0.9447,
         0.7629]])


In [56]:
import argparse
import os
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from torch_sparse import SparseTensor

# parser = argparse.ArgumentParser(description='')
# parser = argparse.ArgumentParser()
# parser.add_argument('--seed', type=int, default=666, help='Random seed.')
# parser.add_argument('--dataset', type=str, default='citeseer', choices=['bitcoin_alpha','ca_grqc','wikivote','cora_ml','citeseer','blogcatalog'], help='dataset')
# parser.add_argument('--trial', type=int, default=1, choices=[1,2,3,4,5], help='trial')
# parser.add_argument('--size', type=int, default=10, choices=[10,30], help='target nodes size')
# parser.add_argument('--B', type=int, default=35, choices=[35], help='budget B')
# parser.add_argument('--device', type=str, default='cuda', choices=['cuda','cpu'], help='device')

# args = parser.parse_args()





#target_node_lst = np.loadtxt(root_dir + '/dataset/'+args.dataset+'/' + 'rand'+ str(args.size) + '_' + str(args.trial) + '.txt').astype('int')

#mod_dir = root_dir + '/dataset/'+args.dataset+'/GradMaxSearch/' + 'rand'+ str(args.size) + '_' + str(args.trial)

# try:
#     os.makedirs(mod_dir)
# except:
#     pass

class multiple_AS(nn.Module):
    def __init__(self, target_lst, n_node, device):
        """
            target_lst (numpy.ndarray): The target list to be initialized.
            n_node (int): The number of nodes.
            device (str): The device to be used.

        """
        super().__init__()
        self.lst = target_lst
        self.n = n_node
        self.device = device
    
    def adjacency_matrix(self, tri):
        A = torch.sparse_coo_tensor(tri[:,:2].T, tri[:,2], size=[self.n,self.n]).to_dense()
        A = A + A.T - torch.diag(torch.diag(A)) # symmetric.
        return A
    
    def sparse_matrix_power(self, A, tau):
        A_sp = A.to_sparse()
        A_sp = torch.sparse_coo_tensor(A_sp.indices(), A_sp.values(), size=[self.n,self.n])
        return torch.sparse.mm(torch.sparse.mm(A_sp, A_sp), A_sp).to_dense()
    
    def extract_NE(self, A):    # Extract node and edge information based on adjecency matrix
        N = torch.sum(A, 1)
        E = torch.sum(A, 1) + 0.5 * torch.diag(self.sparse_matrix_power(A, 3)).T
        N = N.reshape(-1,1)
        E = E.reshape(-1,)
        return N, E
    
    def OLS_estimation(self, N, E):
        """
        OLS estimation function that calculates the Ordinary Least Squares estimate.
        
        Parameters:
            N (tensor): Input tensor for independent variable N
            E (tensor): Input tensor for dependent variable E
        
        Returns:
            tensor: Tensor result of the OLS estimation
        """
        logN = torch.log(N + 1e-20)
        logE = torch.log(E + 1e-20)
        logN1 = torch.cat((torch.ones((len(logN),1)).to(self.device), logN), 1)
        return torch.linalg.pinv(logN1) @ logE
        
    def forward(self, tri): # Calculate the loss function
        A = self.adjacency_matrix(tri)
        N, E = self.extract_NE(A)
        theta = self.OLS_estimation(N, E)
        b = theta[0]
        w = theta[1]
        tmp = 0.
        for i in range(len(self.lst)):
            tmp += (torch.exp(b) * (N[self.lst[i]]**w) - E[self.lst[i]])**2
        return tmp
    
    def true_AS(self, tri): # Calculate the true active subnetworkscore by using OLS (How influential is this subnetwork)
        A = self.adjacency_matrix(tri)
        N, E = self.extract_NE(A)
        theta = self.OLS_estimation(N, E)
        b = theta[0]
        w = theta[1] 
        tmp = 0.
        for i in range(len(self.lst)):
            tmp += (torch.max(E[self.lst[i]],torch.exp(b)*(N[self.lst[i]]**w))\
                   /torch.min(E[self.lst[i]],torch.exp(b)*(N[self.lst[i]]**w)))*\
                    torch.log(torch.abs(E[self.lst[i]]-torch.exp(b)*(N[self.lst[i]]**w))+1)
        return tmp

def Poison_attack(model, triple, B):
    triple_copy = triple.copy()
    #np.savetxt(mod_dir+'/triple_mod_'+str(0)+'.txt',triple_copy,fmt='%d')
    triple_torch = Variable(torch.from_numpy(triple_copy), requires_grad = True)
    AS = []
    perturb = []
    AS.append(model.true_AS(triple_torch).data.numpy()[0])
    print('initial active subnetworkscore:', model.true_AS(triple_torch).data.numpy()[0])
    

    print("--- In attack stage ---")

    for i in range(1,B+1):
        loss = model.forward(triple_torch)
        loss.backward()
        
        tmp = triple_torch.grad.data.numpy()


        grad = np.concatenate((triple_torch[:,:2].data.numpy(),tmp[:,2:]),1)
        

        v_grad = np.zeros((len(grad),3))

        for j in range(len(grad)):
            v_grad[j,0] = grad[j,0]
            v_grad[j,1] = grad[j,1]
            if triple_copy[j,2] == 0 and grad[j,2] < 0:
                v_grad[j,2] = grad[j,2]
            elif triple_copy[j,2] == 1 and grad[j,2] > 0:
                v_grad[j,2] = grad[j,2]
            else:
                continue
        v_grad = v_grad[np.abs(v_grad[:,2]).argsort()]
        # attack w.r.t gradient information.
        K = -1

        while v_grad[K][:2].astype('int').tolist() in perturb:
            K -= 1
            
            
        # do not delete edge from singleton.
        while v_grad[int(K)][2] > 0 and \
             (model.adjacency_matrix(triple_torch).data.numpy()[int(v_grad[int(K)][0])].sum() <= 1 or \
              model.adjacency_matrix(triple_torch).data.numpy()[int(v_grad[int(K)][1]) ].sum() <= 1):
            K -= 1
        
        target_grad = v_grad[int(K)]
        print(K, target_grad)
        target_index = np.where(np.all((triple[:,:2] == target_grad[:2]), axis = 1) == True)[0][0]
        triple_copy[target_index,2] -= np.sign(target_grad[2])
        #np.savetxt(mod_dir+'/triple_mod_'+str(i)+'.txt',triple_copy,fmt='%d')
        triple_torch = Variable(torch.from_numpy(triple_copy), requires_grad = True)
        perturb.append([int(target_grad[0]),int(target_grad[1])])
        true_AScore = model.true_AS(triple_torch).data.numpy()[0]
        AS.append(true_AScore)
        print('iter', i, 'active subnetworkscore:', true_AScore)
    AS = np.array(AS)    

    # From Local Dice: 
    # # 5. build perturbed adjacency
    # A_rows, A_cols, A_vals = adj.coo()
    # A_idx = torch.stack([A_rows, A_cols], dim=0)

    # is_before = A_rows < model.lst
    # is_after = A_rows > node_idx

    # i_col = (
    #     torch.cat([neighbors_idx[~delete_neighbors_mask], add_neighbors_idx], dim=0)
    #     .sort()
    #     .values
    # )
    # i_row = torch.full_like(i_col, node_idx)
    # i_idx = torch.stack([i_row, i_col], dim=0)
    # i_val = torch.ones(i_idx.shape[1])

    # A_idx = torch.cat((A_idx[:, is_before], i_idx, A_idx[:, is_after]), dim=-1)
    # A_weights = torch.cat((A_vals[is_before], i_val, A_vals[is_after]), dim=-1)

    # triple_sparse = SparseTensor.from_edge_index(
    #         A_idx, A_weights, (self.n, self.n)
    #     )
    
    # return triple_sparse, AS

    # Assuming triple_torch_data is a tuple of three tensors representing the COO format
    # Create a sparse tensor from the COO format data

    #convert to sparse

    sparse_tensor = triple_torch.to_sparse()

    return triple_torch, AS



In [73]:

from pygod.detector import DOMINANT
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from Utils.graph_utils import prepare_graph, adj_matrix_sparse_coo_to_dense
from Utils.poison_utils import poison_n_nodes
from Utils.experiment_results import Experiment
from Poison.local_dice import LocalDICE
from pygod.metric import eval_roc_auc
import torch
from pygod.utils import load_data
import copy
from typing import Tuple, List, Any

import sys

from sklearn.metrics import (
roc_auc_score,
average_precision_score,
f1_score
)


sys.path.append('..')
#data = Planetoid("./data/Cora", "Cora", transform=T.NormalizeFeatures())[0]
data = load_data("inj_cora")
y_binary: List[int] = data.y.bool()
print(y_binary)

detector = DOMINANT(hid_dim=64, num_layers=4, epoch=100)
detector.fit(data)

pred, score, prob, conf = detector.predict(data,
                                    return_pred=True,
                                    return_score=True,
                                    return_prob=True,
                                    return_conf=True)
experiment_before_poison = Experiment(data=data, pred=pred, prob=prob, score=score, conf=conf)

node_attr, adj, labels = prepare_graph(data)


amount_of_nodes = data.num_nodes

# 'triple' is a list that will store the perturbed triples during the poisoning process.
# Each triple represents an edge modification in the form of (node1, node2, edge_label).


dense_adj = adj.to_dense()  #Fill in zeroes where there are no edges


print("Create triples...")
triple = []
for i in range(amount_of_nodes):
    for j in range(i+1,amount_of_nodes):
        triple.append([i,j,dense_adj[i,j]])  #Fill with 0, then insert actual after


triple = np.array(triple)

print("Done creating triples...")


# flat_adj = torch.flatten(dense_adj)    #Flatten the matrix to 1 dim (list)
# #print(dense_adj)




# print("Create triples...")
# trple = []
# for i in range(amount_of_nodes):
#     for j in range(i+1,amount_of_nodes):
#         triple.append([i,j,flat_adj[amount_of_nodes*i,j]])  #Fill with 0, then insert actual after

# print("Done creating triples...")

# print(triple)

# triple = np.array(triple)



target_node_lst = np.array([2,3,5])

print("Making model...")
model = multiple_AS(target_lst = target_node_lst, n_node = amount_of_nodes, device = 'cpu')

budget = 2

print("Starting attack...")

adj_adversary, AS = Poison_attack(model, triple, budget)
#np.savetxt(mod_dir+'/AS.txt',AS)
#plt.plot(AS)

# ld = LocalDICE(adj=adj, attr=node_attr, labels=labels, target_node_id=2)
# # Todo: Vary perburtations
# ld, node_idxs = poison_n_nodes(ld, 10, 75)
# print("These are the node_idxs: ${}".format(node_idxs))


# Convert adversary adjecency matrix into compatible dense tensor
#adj_adversary = adj_matrix_sparse_coo_to_dense(ld.adj_adversary)
#adj_adversary = adj_matrix_sparse_coo_to_dense(A_mod)


tensor([False, False, False,  ..., False, False, False])
node_attrs
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
adj
SparseTensor(row=tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
             col=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
             val=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
             size=(2708, 2708), nnz=11054, density=0.15%)
labels
tensor([0, 0, 0,  ..., 0, 0, 0])
Create triples...
Done creating triples...
Making model...
Starting attack...
initial active subnetworkscore: 1.8646916462163183
--- In attack stage ---
-2 [   2.         1986.            3.76079248]
iter 1 active subnetworkscore: 1.4100335427135857
-2 [   2.         1666.            2.22937198]
iter 2 active subnetworkscore: 0.9340035729301503


In [74]:

print("Running model on poisoned data...")

# Create Edge Index'
#edge_index = torch.tensor([torch.tensor([]), torch.tensor([])], dtype=torch.long)
edge_index = torch.tensor([[],[]])
print(edge_index.shape)

#adj_adversary = adj_adversary.type(torch.int64)

print(adj_adversary.shape)
# Transpose it to make shape compatible
transposed_adj_adversary = torch.transpose(adj_adversary, 0, 1)

for i in range(len(adj_adversary)):
    if(adj_adversary[i][2] != 0):
        #Add edge to edge index 
        edge_index = torch.cat((edge_index, transposed_adj_adversary[:2, i:i+1]), -1)
        # edge_index[0] = torch.cat((edge_index[0], torch.tensor([adj_adversary[i][0]])))
        # edge_index[1] = torch.cat((edge_index[1], torch.tensor([adj_adversary[i][1]])))
        # Dataset uses edges both ways so add reverse edge as well
        edge_index = torch.cat((edge_index, torch.flip(transposed_adj_adversary[:2, i:i+1], dims=[0])), -1)
        # edge_index[0] = torch.cat((edge_index[0], torch.tensor([adj_adversary[i][1]])))
        # edge_index[1] = torch.cat((edge_index[1], torch.tensor([adj_adversary[i][0]])))
    
    
print("--- Data compare ---")

print(data)


data_after_poison = copy.deepcopy(data)

edge_index = edge_index.type(torch.int64)

data_after_poison.edge_index = edge_index


detector_poisoned = DOMINANT(hid_dim=64, num_layers=4, epoch=100)
detector_poisoned.fit(data_after_poison)

pred_after, score_after, prob_after, conf_after = detector_poisoned.predict(data_after_poison,
                                        return_pred=True,
                                        return_score=True,
                                        return_prob=True,
                                        return_conf=True)
experiment_after_poison = Experiment(data=data_after_poison, pred=pred_after, score=score_after, prob=prob_after, conf=conf_after)


print("auc before poison:")
print(roc_auc_score(y_binary, score))
print("auc after poison:")
print(roc_auc_score(y_binary, score_after))
#print(roc_auc_score(data_inj.y.detach().numpy(), score_inj.detach.numpy()))


#return experiment_before_poison, experiment_after_poison #, node_idxs, y_binary

#return pred, score, prob, conf, pred_after, score_after, prob_after, conf_after

Running model on poisoned data...
torch.Size([2, 0])
torch.Size([3665278, 3])


--- Data compare ---
Data(x=[2708, 1433], edge_index=[2, 11060], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], s=[2708, 2708])
auc before poison:
0.7692494219816163
auc after poison:
0.7666102746292224
